In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 開始使用 Vertex AI Codey API - 程式碼產生

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/language/code/code_generation.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory 標誌"><br>在 Colab 上執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/code/code_generation.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub 標誌"><br>在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/code/code_generation.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI 標誌"><br>在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者 | [Lavi Nigam](https://github.com/lavinigam-gcp), [Polong Lin](https://github.com/polong-lin) |


## 概觀

本筆記本將提供有關使用 [Codey 模型](https://cloud.google.com/vertex-ai/docs/generative-ai/code/code-models-overview) 執行程式碼產生的簡介。用於程式碼產生的 Codey (code-bison) 是一種基礎模型，可以根據自然語言描述產生程式碼。它可以用來建立函式、網頁、單元測試和其他類型的程式碼。用於程式碼產生的 Codey 由程式碼產生 Codey API 支援，而程式碼產生 Codey API 是 PaLM API 系列的一部份。

### Vertex AI PaLM API
[在 2023 年 5 月 10 日發布](https://cloud.google.com/vertex-ai/docs/generative-ai/release-notes#may_10_2023) 的 Vertex AI PaLM API 由 [PaLM 2](https://ai.google/discover/palm2) 提供技術支援。

### 使用 Vertex AI PaLM API

你可以使用下列方式與 Vertex AI PaLM API 互動：

* 使用 [Generative AI Studio](https://cloud.google.com/generative-ai-studio) 進行快速測試和指令產生。
* 在 Cloud Shell 中使用 cURL 指令。
* 在 Jupyter 筆記本中使用 Python SDK

本筆記本著重於使用 Python SDK 來呼叫 Vertex AI PaLM API。如需更多有關無需撰寫程式碼即可使用 Generative AI Studio 的資訊，你可以瀏覽 [使用 UI 說明開始](https://github.com/doggy8088/generative-ai/blob/main/language/intro_vertex_ai_studio.md)


如需更多資訊，請查看 [Vertex AI 生成式 AI 支援文件](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview)。


### 目標

在此教學課程中，你將會學習：

- 設定程式碼生成模型的程式環境
- 為 codey 撰寫基礎提示
- 各種提示設計模式：
	- 程式題目範本
	- SQL 元資料和效能
	- 程式碼最佳化
	- 思考鏈
	- 數次的提示
	- DevOps 範本
	- 網頁範本
- 程式碼生成的最佳作法


### 費用
本教學指南使用 Google
Cloud 的計費元件：

* Vertex AI Generative AI Studio

了解 [Vertex AI 定價](https://cloud.google.com/vertex-ai/pricing)
，並使用 [定價計算器](https://cloud.google.com/products/calculator/)
，根據預計用量產生成本估算。


## 開始使用


### 安裝 Vertex AI SDK


In [ ]:
!pip install google-cloud-aiplatform==1.36.2 --upgrade --user

**僅限 Colab：** 取消以下Cell註解以重新啟動核心或使用按鈕重新啟動核心。對於 Vertex AI Workbench，你可以使用右上方的按鈕重新啟動終端機。


In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### 驗證筆記本環境
* 如果你使用 **Colab** 執行此筆記本，取消註解下方的Cell並繼續。
* 如果你使用 **Vertex AI 工作台** ，請查看[此處](https://github.com/doggy8088/generative-ai/tree/main/setup-env)的設定說明。


In [ ]:
# from google.colab import auth
# auth.authenticate_user()

## Vertex AI PaLM API 模型


Vertex AI PaLM API 讓你能夠測試、自訂和部署稱為 PaLM 的 Google 大型語言模型 (LLM) 實例，如此一來你就能在你應用程式中發揮 PaLM 的功能。

### 模型命名規則
基礎模型名稱有三個組成部分：使用案例、模型大小和版本號。命名慣例的格式如下： 
`<用途>-<模型大小>@<版本號>`

例如，text-bison@002 代表 Bison 文字模型，版本 002。

模型大小如下所示：
- **Bison** ：功能和成本最佳值。
- **Gecko** ：最小型且最便宜的模型，適合簡單的任務。

### 可用的模型

Vertex AI Codey API 目前支援三種模型：

* `code-bison@002`：根據所需的程式碼的自然語言描述來微調生成程式碼的模型。例如，它可以產生函式的單元測試。

* `code-gecko@002`：根據已寫入程式碼中的內容建議程式碼補全的微調模型。

* `codechat-bison@002`：針對聊天機器人對話微調的模型，有助於解答與程式碼相關的問題。

你可以在 [Generative AI Studio 文件](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#foundation_models) 中找到有關這些基礎模型的更多資訊。


### 匯入函式庫


**僅限 Colab：** 取消下方單元格的註解，以初始化 Vertex AI SDK。對於 Vertex AI Workbench，不需要執行此動作。


In [ ]:
# import vertexai

# PROJECT_ID = ""  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
from IPython.display import Markdown, display
from vertexai.language_models import CodeGenerationModel

## 程式碼生成與 code-bison@002


在本筆記本中，你將會使用來自 PaLM API 的程式碼生成模型 (Codey) 是 code-bison@002。此模型經過微調，能夠遵循自然語言指令產生必要的程式碼，適用於各種程式碼任務，例如：

 - 編寫函式
 - 編寫類別
 - 網頁
 - 單元測試
 - Docstring
 - 程式碼翻譯，以及許多其他用途。

當前支援語言包含：
 - C++
 - C#
 - Go
 - GoogleSQL
 - Java
 - JavaScript
 - Kotlin
 - PHP
 - Python
 - Ruby
 - Rust
 - Scala
 - Swift
 - TypeScript

你可以在此找到更進一步的資訊 [here](https://cloud.google.com/vertex-ai/docs/generative-ai/code/code-models-overview)。


### 載入模型


In [ ]:
code_generation_model = CodeGenerationModel.from_pretrained("code-bison@002")

### `code-bison` 的模型參數

你可以針對 `code-bison@002` 使用下列參數，自訂 PaLM API 程式碼產生針對提示的行為：

- `prefix`：表示有意義的程式碼區塊開頭，或描述要產生的程式碼的自然語言提示。
- `temperature`：越高表示程式碼回應越「有創意」。範圍：(0.0 - 1.0，預設 0)。
- `max_output_tokens`：設定輸出中 Token 的最大數量。範圍：(1 - 2048，預設 2048)


### 你好，Codey


你可以測試正在生成之程式碼 [這裡](https://onecompiler.com/)


In [ ]:
prefix = "write a python function to do binary search"

response = code_generation_model.predict(prefix=prefix)

print(response.text)

### 嘗試使用你自己的提示


一些範例：
*撰寫 Go 程式，從文字檔中萃取 IP 位址
*撰寫 Java 程式，用於從地址中萃取郵遞區號
*撰寫標準 SQL 函式，移除字串中所有非字母字元，並將其編碼為 utf-8


In [ ]:
prefix = """write a python function that can do cosine similarity between two vectors,
            named as "calculate_cosine_similarity" and two input arguments "vector1" and "vector2". \
          """

response = code_generation_model.predict(prefix=prefix, max_output_tokens=1024)

print(response.text)

### 提示範本


提示模板在你找到一個可以重複使用且結構化良好的提示構成方法時很有用。這也可以有助於限制自由形式提示的開放性。有很多方式可以實作提示模板，下方只是一個使用 f 字串的範例。透過這樣的方式你可以依照預期的程式碼功能來構成提示。


In [ ]:
language = "python"
prefix = f"""Write a {language} function that can input unsorted array or list and return the sorted array or list.
             it should not use any pre-built function or library.
              """

response = code_generation_model.predict(prefix=prefix, max_output_tokens=1024)

print(response.text)

In [ ]:
language = "python"
file_format = "json"
extract_info = "names"
requirments = """
              - the name should be start with capital letters.
              - There should be no duplicate names in the final list.
              """

prefix = f"""Create a {language} to parse {file_format} and extract {extract_info} with the following requirements: {requirments}.
              """

response = code_generation_model.predict(prefix=prefix, max_output_tokens=1024)

print(response.text)

## 提示設計模式


### 問題說明範本

在問題說明範本中，你可以利用提示範本概念並傳遞你具體的問題說明，而不用關注於語言。語言可以是單獨的參數。你也可以傳遞輸入輸出範例，以確保生成遵循測試使用案例。以下是一些不同語言的範例；你可以看到模型生成多種支援語言的能力。


#### C


In [ ]:
language = "c"
problem_statement = "find the smallest element in an unordered list"

prefix = "write a " + language + " function to " + problem_statement

response = code_generation_model.predict(prefix=prefix)

print(response.text)

#### C++


In [ ]:
language = "cpp"
problem_statement = """Sort an array in one swap whose two elements are swapped and rest are in sorted order \
                      for example: \
                      input: {1, 5, 3, 7, 9}
                      output: {1,3,5,7,9}
                    """

prefix = "write a " + language + " function to " + problem_statement

response = code_generation_model.predict(prefix=prefix)

print(response.text)

#### Clojure


In [ ]:
language = "clojure"
problem_statement = """that takes a string and calculates if its palindrome or not.
                      print the outputs with two example: 'radar' and 'happy'
                    """

prefix = "write a " + language + " function " + problem_statement

response = code_generation_model.predict(prefix=prefix)

print(response.text)

#### Elixir


In [ ]:
language = "elixir"
problem_statement = """print the first non-repeated character from a string.
                      take example of 'Mississippi' and 'hello' as example.
                    """

prefix = "write a " + language + " function " + problem_statement

response = code_generation_model.predict(prefix=prefix)

print(response.text)

#### Erlang


In [ ]:
language = "erlang"
problem_statement = """reverse an array in place. Take input of array.Take input of array and output the reversed array.
                       For example: [1,2,3,4,5] -> [5,4,3,2,1]
                    """

prefix = "write a " + language + " program " + problem_statement

response = code_generation_model.predict(prefix=prefix)

print(response.text)

#### Fortran


In [ ]:
language = "Fortan"
problem_statement = """ to remove duplicate elements from given array.
                    """

prefix = "write a " + language + " program " + problem_statement

response = code_generation_model.predict(prefix=prefix)

print(response.text)

#### Go


In [ ]:
language = "Go"
problem_statement = """that can extract ipv4 addresses from the each line in the log file. use fmt and regexp package.
                        input:
                        03/22 08:51:06 INFO   :...read_physical_netif: index #0, interface VLINK1 has address 129.1.1.1, ifidx 0
                        output:
                        129.1.1.1
                        \n\n
                    """

prefix = "write a " + language + " function " + problem_statement

response = code_generation_model.predict(prefix=prefix)

print(response.text)

### SQL 元數據 & 效能

在 SQL 提示範本中，你可以定義多重標準，例如：問題陳述、表格元數據、程式碼樣式，以及效能期望。傳遞表格元數據很關鍵，因為這可以讓模型產生遵循元數據結構的一致程式碼。確保你的問題陳述非常清晰、簡潔，並且包含所有相關脈絡。


#### SQL


In [ ]:
problem_statement = """
                    You are a developer working on an e-commerce platform.
                    The marketing team has requested a report on the total number of orders and the average order \
                    value for each product category for the past month.
                    Your task is to generate a SQL queries to retrieve the total number of orders and the average order \
                    value for each product category for the orders placed in the:
                    1) past month,
                    2) given data range,
                    3) end of each month for given year,
                    4) christmas and new year's eve.
                    """
table_metadata = """
                 - **Orders:**
                    - `OrderID` (integer)
                    - `ProductID` (integer)
                    - `ProductName` (string)
                    - `Category` (string)
                    - `OrderDate` (date)
                    - `OrderAmount` (decimal)
                """
code_style = """
            Write a SQL query that follows best practices, is readable, and well-commented.
             """

performance_requirement = """
                          Optimize the query for performance considering the potential size of the "Orders" table.
                          Consider using appropriate indexing if necessary.
                          """

prefix = f""" Solve the following: {problem_statement}. The given table metadata is : {table_metadata} .
          Follow the following code style:{code_style} . The following performance requirement is: {performance_requirement} .
          """
response = code_generation_model.predict(prefix=prefix)

print(response.text)

#### BigQuery


In [ ]:
metadata = """
          A table of customer data, with the following columns:

          customer_id: The unique identifier for the customer.
          first_name: The customer's first name.
          last_name: The customer's last name.
          email: The customer's email address.
          phone_number: The customer's phone number.
          country: The customer's country of residence.
          order_history: A JSON object containing the customer's order history, including the following information for each order:
          order_id: The unique identifier for the order.
          order_date: The date the order was placed.
          order_total: The total amount of the order.
          order_items: A list of the items ordered, including the following information for each item:
          item_id: The unique identifier for the item.
          item_name: The name of the item.
          item_quantity: The quantity of the item ordered.
          item_price: The price of the item.

            """
language = "BigQuery"
problem = """solve following queries: \n
            - Total number of orders placed. \n
            - Total amount of money spent on orders. \n
            - Average order value. \n
            - Most popular item ordered (by quantity). \n
            - Most recent order placed. \n
          """
additional_requirment = """
            - The query should be efficient and scalable, as the customer table may contain millions of rows. \n
            - The query should be easy to read and maintain.
            """
prefix = f"""Write a {language} query to {problem}.
          use this as the table metadata: {metadata}.
          Here are some additional requirement for the query: {additional_requirment}.
          Generate each query as a separate query seperated with a comment.
              """
response = code_generation_model.predict(prefix=prefix, max_output_tokens=2000)

print(response.text)

### 程式碼最適化

你可以於程式碼的最佳化提示中定義問題陳述的特定最佳化需求。程式碼模型很擅於遵循具體指示並產生可以滿足使用者特定條件的程式碼。你可以使用下列三種語言進行嘗試，並查看如何針對資料結構、演算法複雜度和可維護性提出具體指示。


#### Haskell


In [ ]:
sample_input = "A list of integers, e.g. [1, 2, 3, 4, 5]"
language = "Haskell"
problem_statement = (
    f"a {language} program that calculates the sum of all the integers in the list."
)
additional_requirement = """
                      - The function should be efficient and recursive.
                      - The function should be polymorphic, so that it can be used to sum lists of any type of number.
                      """
prefix = f"""
        Write {problem_statement}. \n
        Also add example use case that take {sample_input} calling the function generated.
        Here are some additional requirement for the function: {additional_requirement}.
        """
response = code_generation_model.predict(prefix=prefix, max_output_tokens=1000)

print(response.text)

#### Java


In [ ]:
sample_input = """ "(1 + 2) * 3"
              """
language = "Java"
problem_statement = f"a {language} program that evaluates the mathematical expression and prints the result to the console."
additional_requirement = """
                      - The program should handle all valid mathematical expressions, including those with parentheses, operators, and variables.
                      """
prefix = f"""
        Write {problem_statement}. \n
        Use {sample_input} to test the generated code.
        Here are some additional requirement for the function: {additional_requirement}.
        """
response = code_generation_model.predict(prefix=prefix)

print(response.text)

#### JavaScript


In [ ]:
sample_input = """
              A JSON object containing a list of products, each product with the following properties:
              id: The unique identifier for the product.
              name: The name of the product.
              price: The price of the product.
              quantity: The quantity of the product in stock.
              """
language = "JavaScript"
problem_statement = f"""a {language} function that takes the JSON object as input and returns a new JSON object containing the following properties:
                      total_price: The total price of all the products in the list.
                      average_price: The average price of all the products in the list.
                      most_expensive_product: The most expensive product in the list.
                      least_expensive_product: The least expensive product in the list.
                      out_of_stock_products: A list of all the products that are out of stock.
                    """
additional_requirement = """
                      - The function should be efficient and scalable, as the JSON object may contain millions of products.
                      - The function should be easy to read and maintain.
                      """
prefix = f"""
        Write {problem_statement}. \n
        Also add example use case that take {sample_input} calling the function generated.
        Here are some additional requirement for the function: {additional_requirement}.
        """
response = code_generation_model.predict(prefix=prefix)

print(response.text)

### 思考鏈

思考鏈 (CoT) 提示技術可用來引導大型語言模型 (LLM) 透過將任務分解成一系列的自然語言中介推理步驟，來產生程式碼。此種方式已證明與傳統提示方法相比，它能提升產出程式的品質。

若要將 CoT 提示用於產生程式碼，你首先得提供 LLM 一段自然語言說明，宣告你想達成的任務。LLM 接著會產生思考鏈，也就是一系列說明如何解決任務的自然語言步驟。最後，LLM 會使用思考鏈來產生程式碼。

以下為使用 CoT 提示產生 C++ 和 Java 函式的範例，以因應特定使用案例。


#### C++


In [ ]:
language = "C++"
sample_input = """
                17/06/09 20:10:41 INFO slf4j.Slf4jLogger: Slf4jLogger started
                17/06/09 20:10:41 INFO Remoting: Starting remoting
                17/06/09 20:10:41 INFO Remoting: Remoting started; listening on addresses :[akka.tcp://sparkExecutorActorSystem@mesos-slave-07:55904]
                17/06/09 20:10:41 INFO util.Utils: Successfully started service 'sparkExecutorActorSystem' on port 55904.
              """
additional_requirement = """
                      - It should not use regex to find the given line
                      - the solution should be scalable and should scale linearly with additional data
                      - the output should print a flag and port number both.
                      - All the variable should be properly in scope and should be decalred only once.
                      - The varibles should have scope to be called in the main() function.
                      - The code should be easy to read and maintain
                      - The code should have proper typehints and comments
                      """

prefix = f"""
        Prompt 1: What is the problem we are trying to solve?

        Identify the status of the sparkExecutorActorSystem service in network log and output True or False along with the port, if True.

        Prompt 2: What is the language you want to use to solve this problem?
        {language}

        Prompt 3: What are the inputs and outputs of the function?

        Input: Network log
        Output: Boolean value indicating the status of the sparkExecutorActorSystem service and the port, if True

        Prompt 4: What are the steps involved in identifying the status of the sparkExecutorActorSystem service in network log?

        Split the network log into lines.
        Iterate over the lines and search for the line that contains the following string: Successfully started service 'sparkExecutorActorSystem' on port.
        If the line is found, extract the port number and save it in the variable
        Make sure the varibles are scoped to be called in the main() function.
        If the line is not found,
        Return True along with the port number variable.
        Otherwise, return False.
        call the function passing the sample input
        Prompt 5: What is the sample input that can be tested as a test use case?
        {sample_input}

        prompt 6: Any additional expectation from the code logic?
        {additional_requirement}

        Prompt 7: Write the code for the scenario keeping additional expectation and expected language while generation along with the test case.

        """
response = code_generation_model.predict(prefix=prefix)

print(response.text)

#### Java


In [ ]:
prefix = """
        q: What is the input to the function?
        a: The input to the function is a string.

        q: What is the output of the function?
        a: The output of the function is a reversed string.

        q: What are the steps involved in reversing a string?
        a: 1) Iterate over the string from the back.
            2) Add each character to a new string in reverse order.
            3) Return the new string.

        q: Write pseudocode for the function.
        a: function reverse_string(string):
              new_string = ""
              for i in range(len(string) - 1, -1, -1):
                  new_string += string[i]
              return new_string

        q: how would you test the function?
        a: the input "hello" should return "olleh"

        q: write java code for the function follwing all the question-answer pairs.
        """
response = code_generation_model.predict(prefix=prefix, max_output_tokens=2048)

print(response.text)

### 透過使用者旅程和偽碼/初始碼進行小範例

在小範例提示中，你也可以傳遞使用者旅程或偽碼/初始碼範例，讓程式碼產生遵循特定指令。使用者旅程也可以包含範例輸入和輸出資料結構。你也可以透過傳遞一些初始碼，來補充這個範例，讓模型可以依照你想要的結構進行程式碼產生。以下是不同語言的一些範例。


#### Kotlin


In [ ]:
user_journey = """
              A Kotlin developer is working on a new Android app.
              They need to implement a feature that allows users to search for nearby restaurants.
              """
sample_input = """
              A list of restaurants and a search query:
              val restaurants = listOf(
                                Restaurant("The Grill", "123 Main Street", cuisine = "American"),
                                Restaurant("Thai Paradise", "456 Elm Street", cuisine = "Thai"),
                                Restaurant("Little Italy", "789 Pine Street", cuisine = "Italian"),
                              )

              val searchQuery = "Italian"

              """
sample_output = """
          A list of restaurants that match the search query:
          val matchingRestaurants = listOf(
              Restaurant("Little Italy", "789 Pine Street", cuisine = "Italian"),
            )
          """
language = "Kotlin"
problem_statement = f"""a {language} function that takes a list of restaurants and a search query as input and returns  \
                        list of restaurants that match the search query. The search query can be a substring of the restaurant name, \
                        address, or cuisine type.
                    """
additional_requirement = """
                     - The function should be efficient and scalable.
                     - The function should be easy to read and maintain.
                      """
prefix = f"""
        Write {problem_statement}. \n
        Also add example use case that take {sample_input} and {sample_output} calling the function generated.
        Here are some additional requirement for the function: {additional_requirement}.
        """
response = code_generation_model.predict(prefix=prefix)

print(response.text)

#### Rust


In [ ]:
real_world_case = """You are a software engineer at a company that develops trading software.
                     You need to write a Rust program to calculate the moving average of a stock price over a given period of time."""
problem_statement = f"""The program should take two inputs:
                        A vector of stock prices.
                        The period over which to calculate the moving average.
                        The program should output the moving average of the stock price over the given period.
                    """
code_style = "Idiomatic Rust"
algorithmic_complexity = "O(n)"
pseudocode = """
              1) Initialize a variable to store the moving average.
              2) Iterate over the vector of stock prices, adding each price to the moving average.
              3) Divide the moving average by the period to get the average price over the given period.
              4) Return the moving average price.
             """
test_cases = """
            // Test case 1
            let stock_prices = vec![100, 110, 120, 130, 140];
            let period = 3;

            let moving_average = calculate_moving_average(&stock_prices, period);

            assert_eq!(moving_average, 200.0);

            """
sample_input = "[500, 100, 300, 450, 120]"
prefix = f"""Write a Rust program based on a {real_world_case} .
             The problem statement that needs to be addressed is {problem_statement} .
             You can use this pseudocode as an example to generate the code step by step:  {pseudocode} .
             Add an example to call the generated function in main() {sample_input}
             It should follow the code style pattern as {code_style} and should have {algorithmic_complexity} as algorithmic complexity.
             Make sure that the code generated passes the following test cases: {test_cases}
             """
response = code_generation_model.predict(
    prefix=prefix, max_output_tokens=2000, temperature=0.2
)

print(response.text)

#### Scala


In [ ]:
persona = """You are a Scala developer working on a backend service for an e-commerce platform. """
goal = """Your task is to generate Scala code for a data model representing products in the platform's catalog.
          The product data model should include information such as product ID, name, price, and availability.
       """
user_journey = """
              As a developer, your day-to-day tasks often involve designing data models to represent various entities in your application.
              In this scenario, you are tasked with creating a Scala case class for the product data model and a companion object with utility methods.
              """
requirements = """
            1. Create a Scala case class named `Product` with the following fields:
                - `id` (String)
                - `name` (String)
                - `price` (Double)
                - `available` (Boolean)

            2. Implement a companion object for the `Product` case class with the following methods:
                - `create` method that takes parameters for ID, name, price, and availability and returns an instance of the `Product` case class.
                - `format` method that takes a `Product` instance and returns a formatted string representation of the product.

            3. Ensure that the `create` method sets the availability to `true` by default if not provided.
              """
code_structure = """
                You have been provided with a starter code structure. Your task is to complete the code to meet the above requirements. The input code structure is as follows:

                ```scala
                // Starter code
                case class Product(id: String, name: String, price: Double, available: Boolean)

                object Product {
                  // Your generated Scala code for the companion object goes here
                }

                object Main extends App {
                  // Your test cases go here
                }
                """

prefix = f"""{persona} {goal} {user_journey} {requirements} {code_structure} """
response = code_generation_model.predict(
    prefix=prefix, max_output_tokens=2000, temperature=0.2
)

print(response.text)

#### Shell 腳本


In [ ]:
persona = """
          Imagine you are a system administrator responsible for managing a Linux server environment.
          Your daily tasks often involve creating shell scripts to automate various system maintenance and monitoring tasks.
          In this scenario, you are tasked with generating a shell script to automate a common backup task.
          """
user_journey = """
          As a system administrator, you frequently need to create backup scripts to ensure data integrity and disaster recovery.
          Your goal is to generate a simple shell script that backs up a specified directory to a target backup location using the `rsync` command.
              """
requriements = """
          1. Create a shell script named `backup.sh` that takes two command-line arguments:
            - Source directory: The directory to be backed up.
            - Target directory: The directory where the backup should be stored.

          2. The script should use the `rsync` command to perform the backup. The `rsync` command should:
            - Synchronize the contents of the source directory to the target directory.
            - Preserve file permissions and timestamps.
            - Display progress information during the backup.

          3. Add comments to the script to explain its purpose and usage.
              """
starter_code = """
          You have been provided with a starter code structure.
          Your task is to complete the code to meet the above requirements.
          The input code structure is as follows:

          ```bash
          #!/bin/bash

          # Your generated Shell script code goes here
          """

prefix = f"""{persona} {user_journey} {requriements} {starter_code} """
response = code_generation_model.predict(
    prefix=prefix,
    max_output_tokens=2000,
)

print(response.text)

#### Solidity [區塊鏈]


In [ ]:
user_input = """
         - The address of the user withdrawing the tokens
         - The amount of tokens being withdrawn
         """
return_output = """
         - A boolean value indicating whether the withdrawal was successful
         """
requirements = """
          - The code should be written in Solidity using the latest best practices.
          - algorithm should be O(1) time complexity.
          """


prefix = f"""
         Generate a Solidity function called withdraw() that allows users to withdraw tokens from a decentralized exchange (DEX).
         The function should take the following inputs: {user_input} and should return: {return_output}.
         The function should also meet the following requirements: {requirements}
         """
response = code_generation_model.predict(
    prefix=prefix, max_output_tokens=2000, temperature=0.2
)

print(response.text)

#### Verilog


In [ ]:
prefix = """
        Generate Verilog code for a 3-bit adder circuit that performs addition of two 3-bit numbers using only D flip-flops.
        The circuit should have the following inputs:
        a[2:0]
        b[2:0]
        The circuit should have the following outputs:
        sum[2:0]
        carry
        The circuit should implement the following logic:
        The circuit should add the two 3-bit numbers and store the result in the sum output. The carry output should be set to 1 if the addition results in a carry-out, and 0 otherwise.
        Constraints:
        The circuit must use only D flip-flops.
        """
response = code_generation_model.predict(prefix=prefix)

print(response.text)

### DevOps 範本

Codey 也能幫助你產生與 DevOps 相關的程式碼範例。你可以使用下列範本來產生 Docker、Jenkins、GitLab CI、Prometheus 設定檔以及更多此類的程式碼區塊。


#### Docker


In [ ]:
prefix = """
        Generate a Dockerfile for a Python application that:
          * Builds the image from a python:latest base image
          * Exposes the following ports: 8000
          * Installs the following dependencies: pip install flask
          * Sets the working directory to /app
          * Copies the following files to the image: app.py requirements.txt ./
          * Runs the following command on startup: flask run --host=0.0.0.0
        """
response = code_generation_model.predict(prefix=prefix)

print(response.text)

#### Docker 組合


In [ ]:
prefix = """
        Generate a Compose file for the following services: web
          * Define the following networks: default
          * Define the following volumes: ./app:/app
          * Define the following environments: FLASK_APP=app.py
          * Define the following links: none
          * Define the following depends_on relationships: none
        """
response = code_generation_model.predict(prefix=prefix)

print(response.text)

#### Jenkins


In [ ]:
prefix = """
        Objective: Generate a Jenkinsfile for a parametrized pipeline for a Java project.
        Instructions:
        Allow the user to input parameters like "Environment" and "Feature Toggle."
        Use these parameters in the build and test stages.
        Provide default values for parameters.
        """
response = code_generation_model.predict(prefix=prefix)

print(response.text)

#### GitLab CI


In [ ]:
prefix = """
        Generate GitLab CI YAML configuration for a Node.js project.
        Instructions:
          - Assume the project uses Node.js and has unit tests.
          - Include stages for "Build," "Test," and "Deploy."
          - Specify Node.js version and test script.
          - Use GitLab CI variables for sensitive information.
        """
response = code_generation_model.predict(prefix=prefix)

print(response.text)

#### Prometheus 配置


In [ ]:
prefix = """
         Generate YAML configurations for Prometheus to monitor a containerized application.
          Instructions:
            - Specify the job for scraping metrics.
            - Include targets, labels, and metric relabeling as needed.
            - Set up alerting rules for key metrics.
        """
response = code_generation_model.predict(prefix=prefix)

print(response.text)

### 網頁範本

你也可以根據提示生成 HTML 和 CSS 程式碼。在以下範例中，你可以看到如何提及簡單 HTML 頁面及其特性的各種細節。


#### HTML


In [ ]:
prefix = """write a html code for a simple page:
            - Has a button "click me" and it shows how many time user has hit that button as its counter.
            - Add style element that has page header as "My Page Counter Demo" in brown color
            - Everything should be displayed as 'center'.
            - The counter button should be blue by default and when clicked it should be red.
            - The counter value should be in green color.
            - The bottom of the page should display this message "Codey Generated this page" in bold and big font.
"""
response = code_generation_model.predict(prefix=prefix)

print(response.text)

In [ ]:
prefix = """
        Generate a responsive HTML code for a landing page of a new e-commerce website that sells clothes. The landing page should have the following sections:
         - A header with a logo, a navigation bar, and a search bar.
         - A hero section with a large image of a model wearing clothes from the website and a call to action button.
         - A featured products section with a grid of images of the best-selling products on the website.
         - A testimonial section with quotes from satisfied customers.
         - A footer with contact information and social media links.
       """
response = code_generation_model.predict(prefix=prefix)

print(response.text)

## 最佳實務


### 如何撰寫有效的程式碼生成提示

撰寫程式碼生成提示時，務必盡可能清楚明確。提供給模型的資訊越多，它就能更了解你的用意並產生所需的程式碼。

以下是撰寫有效程式碼生成提示的一些秘訣：

* 從清楚且簡潔的任務描述開始，讓模型執行該任務。例如，不要說「產生一個對數字清單排序的函式」，你可以說「產生一個以遞增順序對整數清單排序的 Python 函式」。

* 提供所需輸入和輸出範例。這將有助於模型了解資料格式和預期輸出。例如，你可以提供一個未排序的數字清單，以及相對應的已排序清單。

* 使用自然語言來描述任務。模型經過大量文字和程式碼資料集的訓練，所以它能理解自然語言提示。例如，你可以說「產生一個在 Python 中反轉字串的函式」。


### 如何選擇合適的溫度和最大輸出 Token

溫度參數控制模型輸出的隨機性。更高的溫度會產生更有創意和多樣化的輸出，但它也可能不太準確。較低的溫度會產生更準確的輸出，但它也可能不太有創意。

最大輸出 Token 參數控制模型將生成的最大 Token 數量。這對於限制輸出程式碼的長度或防止模型產生無限迴圈非常有用。

以下是如何選擇合適溫度和最大輸出 Token 的一些秘訣：

* 對需要高精度的任務使用較低的溫度，例如生成用於機器學習模型的程式碼。

* 對需要創意的任務使用較高的溫度，例如生成用於網路應用程式或遊戲的程式碼。

* 使用最大輸出 Token 參數來限制輸出程式碼的長度或防止模型產生無限迴圈。


如何解讀並使用程式碼生成建議

模型產生的程式碼建議並非總是完美無瑕。仔細檢閱產生的程式碼並進行必要的更動非常重要。

以下是一些解讀及使用程式碼生成建議的技巧：

* 檢查輸出程式碼有無語法錯誤。
* 確保輸出程式碼符合你的編碼標準。
* 測試輸出程式碼以確保其能如預期執行。


### 如何避免常見的程式碼產生陷阱

以下是一些應避免的常見程式碼產生陷阱：

* 使用模糊或不明確的提示。你在提示中能越具體，模型就能越好理解你的意圖和產生所需的程式碼。
* 使用過高的溫度。較高的溫度可能導致較不準確和更具創意的輸出。重要的是根據你嘗試執行的任務選擇正確的溫度。
* 未仔細檢閱產生的程式碼。產生的程式碼並非總是完美的。重要的是仔細檢閱產生的程式碼並進行必要的變更。
